# 설정

In [3]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥ 0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import os
import numpy as np

# 깔끔한 그래프 출력을 위해
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 텍스트 분석 실습 - 캐글 Mercari Price Suggestion Challenge

Mercari Price Suggestion Challenge는 캐글에서 진행된 Challenge로서, 일본의 대형 온라인 쇼핑몰인 Mercari사의 제품에 대해 가격을 예측하는 과제다. 제공되는 데이터 세트는 제품에 대한 여러 속성 및 제품 설명 등의 텍스트 데이터로 구성된다. Mercari사는 이러한 데이터를 기반으로 제품 예상 가격을 판매자들에게 제공하고자 한다. 이와 같은 프로세스를 구현하기 위해 판매자는 제품명, 브랜드 명, 카테고리, 제품 설명 등 다양한 속성 정보를 입력하게 되고, ML 모델은 이 속성에 따라 제품의 예측 가격을 판매자에게 자동으로 제공할 수 있다.

제공되는 데이터 세트의 속성은 다음과 같다.

<ul>
    <li><b>train_id</b>: 데이터 id</li>
    <li><b>name</b>: 제품명</li>
    <li><b>item_condition_id</b>: 판매자가 제공하는 제품 상태</li>
    <li><b>category_name</b>: 카테고리 명</li>
    <li><b>brand_name</b>: 브랜드 이름</li>
    <li><b>price</b>: 제품 가격. 예측을 위한 타깃 속성</li>
    <li><b>shipping</b>: 배송비 무료 여부. 1이면 무료(판매자가 지불), 0이면 유료(구매자 지불)</li>
    <li><b>item_description</b>: 제품에 대한 설명</li>
</ul>

이들 중 price가 예측해야 할 타깃 값이다. 회귀로 피처를 학습한 뒤 price를 예측하는 문제다. 이번 Mercari Price Suggestion이 기존 회귀 예제와 다른 점은 item_description과 같은 텍스트 형태의 비정형 데이터와 다른 정형 속성을 같이 적용해 회귀를 수행한다는 점이다.

## 데이터 전처리

먼저 필요한 라이브러리와 함께 mercari_train.tsv 데이터를 DataFrame으로 로딩하고, 데이터를 간략하게 살펴보겠다.

In [20]:
import py7zr
with py7zr.SevenZipFile('./mercari-price-suggestion-challenge/train.tsv.7z', mode='r') as z:
    z.extractall()

ModuleNotFoundError: No module named 'py7zr'

In [15]:
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

mercari_df = pd.read_csv('./mercari-price-suggestion-challenge/train.tsv.7z', sep='\t', encoding='cp949')
print(mercari_df.shape)
mercari_df.head(3)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbc in position 2: invalid start byte